# Extracting & Preparing data- New York CityBike2022 

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json

In [2]:
import sys
!{sys.executable} -m pip install python-dotenv


# Unzipping & Concactonate all months files of 2022 Citybike

Multistep workflow is needed to prepare the files.
Loops mehtod will be used for such complex workflow and not List comprehension.

unzips each month

finds all Excel/CSV files

concatenates them into one monthly DataFrame

saves each month as a single file

finally merges all months into one big 2022 file

In [2]:
# step 1  create a list of all zipped files

from dotenv import load_dotenv
load_dotenv()
base_path = os.getenv("BASE_PATH")

# Create output folders 
monthly_output = os.getenv("MONTHLY_OUTPUT")
final_output = os.getenv("FINAL_OUTPUT")

# List all zip files (each month)
zip_files = [f for f in os.listdir(base_path) if f.endswith(".zip")]
zip_files


['202201-citibike-tripdata.zip',
 '202202-citibike-tripdata.zip',
 '202203-citibike-tripdata.zip',
 '202204-citibike-tripdata.zip',
 '202205-citibike-tripdata.zip',
 '202206-citibike-tripdata.zip',
 '202207-citibike-tripdata.zip',
 '202208-citibike-tripdata.zip',
 '202209-citibike-tripdata.zip',
 '202210-citibike-tripdata.zip',
 '202211-citibike-tripdata.zip',
 '202212-citibike-tripdata.zip']

In [5]:
# Step 2 — Unzip each month into its own folder
import zipfile

for zip_file in zip_files:
    month_name = zip_file.split("-")[0]  # e.g. "202201"
    month_folder = os.path.join(base_path, month_name)

    # Create folder for extracted files
    os.makedirs(month_folder, exist_ok=True)

    # Extract ZIP
    with zipfile.ZipFile(os.path.join(base_path, zip_file), 'r') as z:
        z.extractall(month_folder)

print("All ZIP files extracted.")


All ZIP files extracted.


In [ ]:
# Step 3 — Read all CSV/Excel files for each month

monthly_dfs = []  # store each month's combined DataFrame

for zip_file in zip_files:
    month_name = zip_file.split("-")[0]
    month_folder = os.path.join(base_path, month_name)

    # Find all CSV/Excel files
    files = [f for f in os.listdir(month_folder) if f.endswith((".csv", ".xlsx"))]

    month_df_list = []

    for file in files:
        file_path = os.path.join(month_folder, file)

        if file.endswith(".csv"):
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)

        month_df_list.append(df)

    # Combine all files for that month
    monthly_df = pd.concat(month_df_list, ignore_index=True)

    # Save monthly combined file
    monthly_df.to_csv(os.path.join(monthly_output, f"{month_name}_combined.csv"), index=False)

    # Store for final merge
    monthly_dfs.append(monthly_df)

print("Monthly files processed and saved.")


In [ ]:
len(monthly_dfs)

#### Important Note:
step 3 should be inside the loop that started in step 2.
carfule with the spacing in writing the code in separate cells.
Alternatively write all steps in one cell code , one loop

len(monthly_dfs) # this can be used to test if it performed the loop on all 12 months, 


In [ ]:
monthly_dfs[-3].head()


In [3]:
# Path to your folder containing monthly CSVs
folder_path = r"C:\Users\analy\Documents\Case-Study5--NewYork_Bike-sharing-service_Stategic_Analysis\Data\monthly_output"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Load each CSV into a DataFrame
monthly_dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]

# Combine all months
full_2022_df = pd.concat(monthly_dfs, ignore_index=True)

full_2022_df.head()

C:\Users\analy\AppData\Local\Temp\ipykernel_25060\3659969719.py:8: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
C:\Users\analy\AppData\Local\Temp\ipykernel_25060\3659969719.py:8: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
C:\Users\analy\AppData\Local\Temp\ipykernel_25060\3659969719.py:8: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
C:\Users\analy\AppData\Local\Temp\ipykernel_25060\3659969719.py:8: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  monthly_dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
C:\Users

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BFD29218AB271154,electric_bike,2022-01-21 13:13:43.392,2022-01-21 13:22:31.463,West End Ave & W 107 St,7650.05,Mt Morris Park W & W 120 St,7685.14,40.802117,-73.968181,40.804038,-73.945925,member
1,7C953F2FD7BE1302,classic_bike,2022-01-10 11:30:54.162,2022-01-10 11:41:43.422,4 Ave & 3 St,4028.04,Boerum Pl\t& Pacific St,4488.09,40.673746,-73.985649,40.688489,-73.991160,member
2,95893ABD40CED4B8,electric_bike,2022-01-26 10:52:43.096,2022-01-26 11:06:35.227,1 Ave & E 62 St,6753.08,5 Ave & E 29 St,6248.06,40.761227,-73.960940,40.745168,-73.986831,member
3,F853B50772137378,classic_bike,2022-01-03 08:35:48.247,2022-01-03 09:10:50.475,2 Ave & E 96 St,7338.02,5 Ave & E 29 St,6248.06,40.783964,-73.947167,40.745168,-73.986831,member
4,7590ADF834797B4B,classic_bike,2022-01-22 14:14:23.043,2022-01-22 14:34:57.474,6 Ave & W 34 St,6364.10,5 Ave & E 29 St,6248.06,40.749640,-73.988050,40.745168,-73.986831,member


In [ ]:
# Step 4 — Combine all months into one full-year dataset
# ---------------------------------------------------------

full_2022_df = pd.concat(monthly_dfs, ignore_index=True)

In [6]:
full_2022_df.shape

(29838806, 13)

In [ ]:
full_2022_df.head()

In [4]:
# Save
# Build the final file path using your hidden FINAL_OUTPUT variable
final_path = os.path.join(final_output, "citibike_2022_full.csv")

# Save as pickle instead of CSV
full_2022_df.to_csv(final_path)

print(f"\nFinal 2022 dataset saved to: {final_path}")


Final 2022 dataset saved to: C:/Users/analy/Documents/Case-Study5--NewYork_Bike-sharing-service_Stategic_Analysis/Data/final_output\citibike_2022_full.csv
